This notebook will use the robin_stocks.robinhood.export functions which write all orders to a csv file, then we will parse them and attempt to pair the opening and closing orders.  This is a WORK IN PROGRESS.  the cells for AAPL orders appear to be working.  

Install the required libraries

In [1]:
%pip install robin_stocks pandas numpy ipykernel


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Log on to the robinhood API endpoint, and create the output directory if it doesn't exist

In [2]:
import robin_stocks.robinhood as r
import pandas as pd
import os  
# Prompt for email address
email = input("Please enter your Robinhood email address: ")

login = r.login(email)

# Ensure the output directory exists
output_dir = '../output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir) 


Now lets get all the options orders.

In [84]:
# # Get all option orders
r.export_completed_option_orders('/', file_name='../output/options_output.csv')

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...


This groups spread order by order_created_at and then calculates total cost based on debit and credit direction.  Spreads orders show up multiple time (number of legs) with the total price and quantity listed on each order, so we group them and only sum them up once per opening and closing spread order

In [3]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/options_output.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])
df['order_created_at'] = df['order_created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filter for the year 2024
#df = df[df['order_created_at'].str.startswith('2024')]

# Select only the columns we want
df = df[['order_created_at', 'chain_symbol', 'expiration_date', 
         'strike_price', 'option_type', 'direction', 'order_quantity', 
         'order_type', 'opening_strategy', 'closing_strategy', 'price', 'order_quantity']]

# Group by 'order_created_at' and aggregate
aggregated_df = df.groupby('order_created_at').agg({
    'chain_symbol': 'first',
    'expiration_date': 'first',
    'strike_price': lambda x: list(sorted(x, key=abs, reverse=True)),  # Create a list of strike price
    'option_type': 'first',
    'direction': 'first',
    'opening_strategy': 'first',
    'closing_strategy': 'first',
    'price': 'first',
    'order_quantity': 'first'
}).reset_index()

# Sort the DataFrame
aggregated_df = aggregated_df.sort_values(by=['chain_symbol', 'expiration_date'])

# Calculate cost
aggregated_df['cost'] = aggregated_df['price'] * aggregated_df['order_quantity'] * 100

# Adjust the cost based on the direction
aggregated_df['cost'] = aggregated_df.apply(
    lambda x: -x['cost'] if x['direction'] == 'debit' else x['cost'],
    axis=1
)      

total_option_cost = aggregated_df['cost'].sum()
print(f"Total option cost: ${total_option_cost:.2f}")
print(aggregated_df['order_created_at'].count())
print(aggregated_df['order_quantity'].sum())

# Save the results
aggregated_df.to_csv('../output/options_orders_parsed.csv', index=False)

# Count rows where 'opening_strategy' is not None
total_opening_strategy_count = aggregated_df['opening_strategy'].notna().sum()
total_closing_strategy_count = aggregated_df['closing_strategy'].notna().sum()


# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# spread_closing_count = df['closing_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# spread_opening_count = df['opening_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# non_spread_closing_count = total_closing_strategy_count - spread_closing_count

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# non_spread_opening_count = total_opening_strategy_count - spread_opening_count
# Calculate total order quantity for opening_strategy
total_opening_quantity = aggregated_df['order_quantity'][aggregated_df['opening_strategy'].notna()].sum()
print(f"Total opening strategy quantity: {total_opening_quantity}")

# Calculate total order quantity for closing_strategy
total_closing_quantity = aggregated_df['order_quantity'][aggregated_df['closing_strategy'].notna()].sum()
print(f"Total closing strategy quantity: {total_closing_quantity}")


#Print the counts
print(f"Opening strategy rows: {total_opening_strategy_count}")
print(f"Closing strategy rows: {total_closing_strategy_count}")
# print(f"Opening strategy rows with 'spread' or 'iron': {spread_opening_count}")
# print(f"Closing strategy rows with 'spread' or 'iron': {spread_closing_count}")
# print(f"Opening strategy rows not with 'spread' or 'iron': {non_spread_opening_count}")
# print(f"Closing strategy rows not with 'spread' or 'iron': {non_spread_closing_count}")

Total option cost: $-3247.00
878
1625.0
Total opening strategy quantity: 1008.0
Total closing strategy quantity: 618.0
Opening strategy rows: 544
Closing strategy rows: 335


this is no longer working after I've changed the strike column to be a list

# AAPL Spread Order Analysis

This cell analyzes AAPL spread orders by:

1. Filtering for AAPL spread/iron condor orders
2. Pairing opening orders with corresponding closing orders
3. Handling partial closings and multiple closing orders per opening order
4. Calculating total opened and closed quantities
5. Identifying unpaired (still open) orders

The analysis provides:
- Count of paired and unpaired orders
- Detailed view of first few paired orders (open and close details)
- List of unpaired (open) orders
- Summary of total opened, closed, and remaining open quantities

This helps in understanding the current state of AAPL spread positions, including fully closed, partially closed, and still open trades.

In [14]:
# Assuming we're using the aggregated_df from the previous cell

# Filter for AAPL spread orders
aapl_spreads = aggregated_df[
    (aggregated_df['chain_symbol'] == 'AAPL') & 
    (aggregated_df['opening_strategy'].str.contains('spread|iron', case=False, na=False) | 
     aggregated_df['closing_strategy'].str.contains('spread|iron', case=False, na=False))
]

# Sort by expiration date and strike price
# aapl_spreads = aapl_spreads.sort_values(['expiration_date', 'strike_price'])

# Initialize lists to store paired and unpaired orders
paired_orders = []
unpaired_opens = []

# Iterate through the orders
for _, order in aapl_spreads.iterrows():
    if pd.notna(order['opening_strategy']):
        # This is an opening order
        remaining_quantity = order['order_quantity']
        matching_closes = []

        potential_closes = aapl_spreads[
            (aapl_spreads['closing_strategy'].notna()) &
            (aapl_spreads['expiration_date'] == order['expiration_date']) &
            (aapl_spreads['strike_price'] == order['strike_price']) &
            (aapl_spreads['option_type'] == order['option_type'])
        ]

        for _, close_order in potential_closes.iterrows():
            if remaining_quantity > 0:
                matched_quantity = min(remaining_quantity, close_order['order_quantity'])
                matching_closes.append((close_order, matched_quantity))
                remaining_quantity -= matched_quantity

        if matching_closes:
            paired_orders.append((order, matching_closes))
            for close_order, _ in matching_closes:
                aapl_spreads = aapl_spreads[aapl_spreads.index != close_order.name]
        else:
            unpaired_opens.append(order)

    aapl_spreads = aapl_spreads[aapl_spreads.index != order.name]

# Print results
print(f"Paired orders: {len(paired_orders)}")
print(f"Unpaired opening orders: {len(unpaired_opens)}")

# Display first few paired orders
for i, (open_order, close_orders) in enumerate(paired_orders[:5]):
    print(f"\nPair {i+1}:")
    print(f"Open: {open_order['order_created_at']} - {open_order['strike_price']} - {open_order['opening_strategy']} - Quantity: {open_order['order_quantity']}")
    total_closed = sum(close_order[1] for close_order in close_orders)
    print(f"Closed: {total_closed} out of {open_order['order_quantity']}")
    for j, (close_order, matched_quantity) in enumerate(close_orders):
        print(f"  Close {j+1}: {close_order['order_created_at']} - {close_order['strike_price']} - {close_order['closing_strategy']} - Matched Quantity: {matched_quantity}")

# Display unpaired opening orders
print("\nUnpaired opening orders:")
for i, order in enumerate(unpaired_opens[:5]):
    print(f"{i+1}: {order['order_created_at']} - Exp: {order['expiration_date']} - {order['strike_price']} - {order['opening_strategy']} - Quantity: {order['order_quantity']}")

# Calculate total opened and closed quantities
total_opened = sum(order['order_quantity'] for order, _ in paired_orders) + sum(order['order_quantity'] for order in unpaired_opens)
total_closed = sum(sum(close_order[1] for close_order in close_orders) for _, close_orders in paired_orders)

print(f"\nTotal opened quantity: {total_opened}")
print(f"Total closed quantity: {total_closed}")
print(f"Remaining open quantity: {total_opened - total_closed}")

ValueError: ('Lengths must match to compare', (11,), (2,))

# AAPL Spread Order Analysis

This cell analyzes AAPL spread orders by:

1. Filtering for AAPL spread/iron condor orders
2. Pairing opening orders with corresponding closing orders
3. Handling partial closings and multiple closing orders per opening order
4. Calculating total opened and closed quantities
5. Identifying unpaired (still open) orders

The analysis provides:
- Count of paired and unpaired orders
- Detailed view of first few paired orders (open and close details)
- List of unpaired (open) orders
- Summary of total opened, closed, and remaining open quantities

For unpaired opening orders, it searches for potential closing orders of individual legs by matching:
- Expiration date
- Any matching strike price
- AAPL as the underlying

This helps in understanding the current state of AAPL spread positions, including fully closed, partially closed, and still open trades, as well as identifying potential leg-by-leg closures.

In [13]:
# Assuming we're using the aggregated_df and aapl_spreads from the previous cell

# Convert strike_price to float when creating aggregated_df
aggregated_df = df.groupby('order_created_at').agg({
    'chain_symbol': 'first',
    'expiration_date': 'first',
    'strike_price': lambda x: [float(s) for s in sorted(x, key=abs, reverse=True)],
    'option_type': 'first',
    'direction': 'first',
    'order_quantity': 'first',
    'order_type': 'first',
    'opening_strategy': 'first',
    'closing_strategy': 'first',
    'price': 'first',
    'order_quantity': 'first'
}).reset_index()

# Function to extract strike prices from the strike_price string
def extract_strikes(strike_string):
    return [float(s.strip('+-')) for s in strike_string.split('/')]

# Process unpaired opening orders
for i, open_order in enumerate(unpaired_opens):
    print(f"\nAnalyzing unpaired order {i+1}:")
    print(f"Open: {open_order['order_created_at']} - Exp: {open_order['expiration_date']} - {open_order['strike_price']} - {open_order['opening_strategy']} - Quantity: {open_order['order_quantity']}")
    
    strikes = extract_strikes(open_order['strike_price'])
    
    potential_closes = aggregated_df[
        (aggregated_df['closing_strategy'].notna()) &
        (aggregated_df['expiration_date'] == open_order['expiration_date']) &
        (aggregated_df['chain_symbol'] == 'AAPL') &
        (aggregated_df['strike_price'].apply(lambda x: any(s in strikes for s in x)))
    ]
    
    if not potential_closes.empty:
        print("Potential closing orders for individual legs:")
        for _, close in potential_closes.iterrows():
            print(f"  Close: {close['order_created_at']} - Exp: {close['expiration_date']} - {close['strike_price']} - {close['closing_strategy']} - Quantity: {close['order_quantity']}")
    else:
        print("No potential closing orders found for individual legs.")

# Calculate and print summary
total_unpaired_quantity = sum(order['order_quantity'] for order in unpaired_opens)
print(f"\nTotal quantity of unpaired opening orders: {total_unpaired_quantity}")
print(f"Number of unpaired opening orders: {len(unpaired_opens)}")


Total quantity of unpaired opening orders: 0
Number of unpaired opening orders: 0


In [4]:
# Get all unique chain symbols
unique_symbols = aggregated_df['chain_symbol'].unique()

# Function to extract strike prices from the strike_price string
def extract_strikes(strike_string):
    return [float(s.strip('+-')) for s in strike_string.split('/')]

# Analyze each symbol
for symbol in unique_symbols:
    print(f"\n--- Analysis for {symbol} ---")
    
    # Filter for the current symbol's spread orders
    symbol_spreads = aggregated_df[
        (aggregated_df['chain_symbol'] == symbol) & 
        (aggregated_df['opening_strategy'].str.contains('spread|iron', case=False, na=False))
    ]
    
    # Find unpaired opening orders
    unpaired_opens = []
    for _, order in symbol_spreads.iterrows():
        closing_order = aggregated_df[
            (aggregated_df['chain_symbol'] == symbol) &
            (aggregated_df['closing_strategy'].str.contains('spread|iron', case=False, na=False)) &
            (aggregated_df['expiration_date'] == order['expiration_date']) &
            (aggregated_df['strike_price'] == order['strike_price'])
        ]
        if closing_order.empty:
            unpaired_opens.append(order)

    # Process unpaired opening orders
    for i, open_order in enumerate(unpaired_opens):
        print(f"\nAnalyzing unpaired order {i+1}:")
        print(f"Open: {open_order['order_created_at']} - Exp: {open_order['expiration_date']} - {open_order['strike_price']} - {open_order['opening_strategy']} - Quantity: {open_order['order_quantity']}")
        
        strikes = set(open_order['strike_price'])
        
        potential_closes = aggregated_df[
            (aggregated_df['closing_strategy'].notna()) &
            (aggregated_df['expiration_date'] == open_order['expiration_date']) &
            (aggregated_df['chain_symbol'] == symbol) &
            (aggregated_df['strike_price'].apply(lambda x: any(s in strikes for s in x)))
        ]
        
        if not potential_closes.empty:
            print("Potential closing orders for individual legs:")
            for _, close in potential_closes.iterrows():
                print(f"  Close: {close['order_created_at']} - Exp: {close['expiration_date']} - {close['strike_price']} - {close['closing_strategy']} - Quantity: {close['order_quantity']}")
        else:
            print("No potential closing orders found for individual legs.")

    # Calculate and print summary for this symbol
    total_unpaired_quantity = sum(order['order_quantity'] for order in unpaired_opens)
    print(f"\nTotal quantity of unpaired opening orders for {symbol}: {total_unpaired_quantity}")
    print(f"Number of unpaired opening orders for {symbol}: {len(unpaired_opens)}")


--- Analysis for AAPL ---


ValueError: ('Lengths must match to compare', (878,), (2,))

In [1]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

unique_symbols = aggregated_df['chain_symbol'].unique()
total_option_event_cost = 0  # Initialize total cash amount

# Create a list to store event data
event_data = []

for symbol in unique_symbols:
    events = r.get_events(symbol)  # Fetch events for the symbol
    for event in events:
        # Ignore rows with total_cash_amount = 0
        if float(event['total_cash_amount']) == 0 :
            continue
        
        # Append event details to the list
        event_data.append({
            'created_at': event['created_at'],
            'chain_symbol': symbol,
            'direction': event['direction'],
            'quantity': event['quantity'],
            'total_cash_amount': event['total_cash_amount'],
            'state': event['state'],
            'underlying_price': event['underlying_price']
        })
        if event['direction'] == 'credit':
            total_option_event_cost += float(event['total_cash_amount'])  # Add for exercise
        elif event['direction'] == 'debit':
            total_option_event_cost -= float(event['total_cash_amount'])  # Subtract for assignment

# Create a DataFrame from the event data
events_df = pd.DataFrame(event_data)

# Save the events DataFrame to a new CSV file
events_df.to_csv('../output/option_events.csv', index=True)


# Print the total cash amount
print(f"Total cash amount: {total_option_event_cost}")


NameError: name 'aggregated_df' is not defined

In [107]:
total =  total_option_cost + total_stock_cost + total_crypto_cost + abs(total_option_event_cost)
print(total)

8919.601299630365
